# Proyecto Final. Parte II
## Análisis del Consumo Literario: Optimización de Propuesta de Valor para Startups de Libros
### Elaborado por: Sarah Moscoso


**Contexto**: El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

**Objetivo**: Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. **Esta información se utilizará para generar una propuesta de valor para un nuevo producto.**

#### OBJETIVOS ESPECÍFICOS

- Encuentra el número de libros publicados después del 1 de enero de 2000.
- Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.
- Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
- Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
- Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

## 1- Conectarse a la base de datos

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


In [2]:
db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, # connection port
 'db': 'data-analyst-final-project-db' # the name of the database
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## 2- Estudia las tablas (imprime las primeras filas)

In [3]:
# Tabla: Books
query = '''
SELECT *
FROM books
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# Tabla: Authors
query = '''
SELECT *
FROM authors
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# Tabla: Publishers
query = '''
SELECT *
FROM publishers
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# Tabla: Ratings
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# Tabla: Reviews
query = '''
SELECT *
FROM reviews
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## 3- Realiza una consulta SQL para cada una de las tareas

## 4- Genera los resultados de cada consulta en el Notebook.

## 5- Describe tus conclusiones para cada una de las tareas.

- **Encuentra el número de libros publicados después del 1 de enero de 2000.**

In [8]:
query1 = '''
SELECT *
FROM books
WHERE publication_date > '2000-01-01';
'''
pd.io.sql.read_sql(query1, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
814,994,408,World War Z: An Oral History of the Zombie War,342,2006-09-12,68
815,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
816,998,201,Year of Wonders,358,2002-04-30,212
817,999,94,You Suck (A Love Story #2),328,2007-01-16,331


Conclusión:

En la primera tarea, se encontraron 819 libros publicados después del 1ro de enero del año 2000

- **Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.**

In [9]:
query2 = '''

SELECT reviews.book_id,
books.title,
AVG(rating) AS avg_rating,
COUNT(review_id) AS num_reviews
FROM reviews 
LEFT JOIN books ON books.book_id = reviews.book_id
LEFT JOIN ratings ON ratings.book_id = reviews.book_id
GROUP BY reviews.book_id, books.title
ORDER BY avg_rating DESC, num_reviews DESC;

'''
pd.io.sql.read_sql(query2, con = engine)

,book_id,title,avg_rating,num_reviews
0,17,A Dirty Job (Grim Reaper #1),5.00,16
1,553,School's Out—Forever (Maximum Ride #2),5.00,12
2,444,Moneyball: The Art of Winning an Unfair Game,5.00,9
3,347,In the Hand of the Goddess (Song of the Liones...,5.00,6
4,642,The Big Bad Wolf (Alex Cross #9),5.00,4
...,...,...,...,...
989,915,The World Is Flat: A Brief History of the Twen...,2.25,12
990,202,Drowning Ruth,2.00,9
991,371,Junky,2.00,4
992,316,His Excellency: George Washington,2.00,4


Conclusión:

Gracias a la consulta de SQL, pudimos contar el número de reseñas que tiene cada libro, así como la calificación promedio por libro. Además se ordenaron de los promedios más altos y también por el número de reseñas más altas.

- **Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).**

In [10]:
query3 = '''
SELECT publishers.publisher_id, publishers.publisher, COUNT(books.book_id) AS publish_sum
FROM publishers
LEFT JOIN books ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > '50'
GROUP BY publishers.publisher_id, publishers.publisher
ORDER BY publish_sum DESC;
'''
pd.io.sql.read_sql(query3, con = engine)

,publisher_id,publisher,publish_sum
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19
...,...,...,...
329,34,Ballantine Books (NY),1
330,225,Plaza y Janés,1
331,138,HarperCollinsPublishers,1
332,245,Random House: Modern Library,1


Conclusión:
Se ha encontrado que la editorial "Penguin Books" es la editorial con más libros publicados, con 42 libros en total. Se hizo un filtro de solo los libros con más de 50 páginas para evitar folletos o cosas que no sean libros. En 2do lugar tenemos a la editorial de "Vintage" con 31 libros.

- **Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.**

In [19]:
query4 = '''
SELECT authors.author, authors.author_id, books.book_id, books.title, AVG(ratings.rating) AS avg_rating
FROM authors
LEFT JOIN books ON books.author_id = authors.author_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY authors.author_id, books.book_id
HAVING COUNT(ratings.rating) > 50
ORDER BY avg_rating DESC;
'''
pd.io.sql.read_sql(query4, con = engine)

,author,author_id,book_id,title,avg_rating
0,J.K. Rowling/Mary GrandPré,236,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634
1,J.R.R. Tolkien,240,722,The Fellowship of the Ring (The Lord of the Ri...,4.391892
2,J.K. Rowling/Mary GrandPré,236,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500
3,Markus Zusak/Cao Xuân Việt Khương,402,656,The Book Thief,4.264151
4,J.K. Rowling/Mary GrandPré,236,300,Harry Potter and the Half-Blood Prince (Harry ...,4.246575
5,Louisa May Alcott,376,399,Little Women,4.192308
6,J.K. Rowling/Mary GrandPré,236,301,Harry Potter and the Order of the Phoenix (Har...,4.186667
7,J.R.R. Tolkien,240,750,The Hobbit or There and Back Again,4.125000
8,Rick Riordan,498,779,The Lightning Thief (Percy Jackson and the Oly...,4.080645
9,William Golding,621,405,Lord of the Flies,3.901408


Conclusión:

Se ha obtenido que los autores J.K. Rowling/Mary GrandPré quines escribieron el libro de Harry Potter and the Prisoner of Azkaban, han sido los los de mayor promedio de calificación con 4.41. Para eso tuvimos que buscar solo entre los autores cuyos libros tuvieran más de 50 reseñas.

- **Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.**

In [32]:
query5 = '''
SELECT COUNT(username) AS tot_us, username
FROM reviews
GROUP BY username
HAVING COUNT(username) > 50;
'''
pd.io.sql.read_sql(query5, con = engine)

,tot_us,username


In [33]:
query5 = '''
SELECT reviews.username, COUNT(*) AS total_reviews
FROM reviews
LEFT JOIN books ON books.book_id = reviews.book_id
GROUP BY reviews.username
HAVING COUNT(*) > 50;
'''
pd.io.sql.read_sql(query5, con = engine)

,username,total_reviews
